In [1]:
# import imageio to open images
import imageio
from PIL import Image
from glob import glob

# import augmenters from imgaug
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapOnImage

# Import matplotlib for visualization
from matplotlib import pyplot as plt
import numpy as np

In [2]:
def getImMk(imgPath, mskPath):
    
    # use imageio library to read the image (alternatively you can use OpenCV cv2.imread() function)
    image = imageio.imread(imgPath)

    # Open image with mask
    pil_mask = Image.open(mskPath)
    
    pil_mask = pil_mask.convert("RGB")

    # Convert mask to binary map
    np_mask = np.array(pil_mask)
    np_mask[np_mask > 150]=255
    np_mask[np_mask<100]=0
    np_mask = np.clip(np_mask, 0, 1)

    # Create segmentation map
    segmap = np.zeros(image.shape, dtype=bool)
    segmap[:] = np_mask
    segmap = SegmentationMapOnImage(segmap, shape=image.shape)
    
    return image, segmap

In [3]:
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.Flipud(0.3), # vertically flip 20% of all images
            # crop images by -5% to 10% of their height/width
            sometimes(iaa.CropAndPad(
                percent=(-0.2, -0.05)
            )),
            sometimes(iaa.Affine(
                scale={"x": (1, 1.4), "y": (1, 1.4)}, # scale images to 80-120% of their size, individually per axis
                #translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
                rotate=(-45, 45), # rotate by -45 to +45 degrees
                order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            )),

            # execute 0 to 3 of the following (less important) augmenters per image
            # don't execute all of them, as that would often be way too strong
            iaa.SomeOf((0, 3),
                [
                    iaa.GaussianBlur((0, 0.8)), # blur images with a sigma between 0 and 3.0
                    iaa.WithBrightnessChannels(iaa.Add((-40, 100))), # change brightness of images (by -10 to 10 of original value)
                    # either change the brightness of the whole image (sometimes
                    # per channel) or change the brightness of subareas
                    iaa.LinearContrast((0.7, 1.2)), # improve or worsen the contrast
                    iaa.PerspectiveTransform(scale=(0.01, 0.1))
                ],
                random_order=False
            )
        ],
        random_order=False
)

In [4]:
def augment(image, segmap, n):
    augmented = [seq(image=image, segmentation_maps=segmap) for _ in range(n)]
    return augmented

## Save results

In [5]:
imgPath = r'C:\Users\A60026184\Desktop\ModelB\Data4Aug5\Train\Images'
mskPath = r'C:\Users\A60026184\Desktop\ModelB\Data4Aug5\Train\Masks'

imgFiles = glob(imgPath + '\\*.*')

outIm = r'C:\Users\A60026184\Desktop\ModelB\Data4Aug5\Train\Images'
outMk = r'C:\Users\A60026184\Desktop\ModelB\Data4Aug5\Train\Masks'

In [7]:
for f in imgFiles:
    
    filename = f.split('\\')[-1]
    print(filename)
    mskFile = glob(mskPath+'\\'+ filename[1:])[0]
    
    image, segmap = getImMk(f, mskFile)
    augmented = augment(image, segmap, 5)

    for i in np.arange(len(augmented)):

        img_res = Image.fromarray(augmented[i][0].astype('uint8'), 'RGB')
        mask_res = Image.fromarray(np.array(augmented[i][1].get_arr()).astype('uint8')*255, 'RGB') #save as 0 or 255

        img_res.save(outIm+"\\"+filename.split('.')[0]+"_A"+str(i)+".jpg")
        mask_res.save(outMk+"\\"+filename.split('.')[0]+"_A"+str(i)+".jpg")
    print('... Saved.')

00_180926_Md_A_M14_1.JPG


C:\Users\A60026184\Anaconda3\envs\colza\lib\site-packages\imgaug\imgaug.py:184: DeprecationWarning: Function `SegmentationMapOnImage()` is deprecated. Use `SegmentationMapsOnImage` instead. (Note the plural 'Maps' instead of old 'Map'.).
  warn_deprecated(msg, stacklevel=3)


... Saved.
00_180926_Md_A_M16_1.JPG
... Saved.
00_180926_Md_A_M21_2.JPG
... Saved.
00_180926_Pr_A_M11_1.JPG
... Saved.
00_181009_Md_A_M17_2.JPG
... Saved.
00_181009_Md_A_M21_2.JPG
... Saved.
00_181010_Pr_D_M11_2.JPG
... Saved.
00_181010_Pr_D_M21_1.JPG
... Saved.
00_181015_Md_A_M02_3.JPG
... Saved.
00_181015_Md_A_M13_2.JPG
... Saved.
00_181016_Pr_A_M05_2.JPG
... Saved.
00_181016_Pr_A_M09_3.JPG
... Saved.
00_181016_Pr_D_M06_1.JPG
... Saved.
00_181016_Pr_D_M20_1.JPG
... Saved.
00_181022_Md_A_M14_1.JPG
... Saved.
00_190913_Pr_A_M12_1.JPG
... Saved.
00_190913_Pr_A_M16_3.JPG
... Saved.
00_190913_Pr_D_M02_1.JPG
... Saved.
00_190913_Pr_D_M16_4.JPG
... Saved.
00_190927_Pr_A_M11_2.JPG
... Saved.
00_190927_Pr_A_M17_2.JPG
... Saved.
00_190927_Pr_A_MA5_2.JPG
... Saved.
00_190927_Pr_D_M01_2.JPG
... Saved.
00_190927_Pr_D_M03_3.JPG
... Saved.
00_190927_Pr_D_M03_4.JPG
... Saved.
00_191007_Pr_A_M03_3.JPG
... Saved.
00_191007_Pr_A_M10_3.JPG
... Saved.
00_191007_Pr_D_M09_2.JPG
... Saved.
00_191007_Pr_D_M1

## Plot results

In [ ]:
img = imgFiles[1]
msk = glob(mskPath+'\\'+ img.split('\\')[-1])[0]

image, segmap = getImMk(img, msk)
augmented = augment(image, segmap, 24)

In [ ]:
#Plot results
results = []

# Import matplotlib for visualization
from matplotlib import pyplot as plt

for im, seg in augmented:
    results.append(seg.draw_on_image(im)[0])
ia.imshow(ia.draw_grid(results, cols=8))

In [ ]:
# Augment images and segmaps.
images_aug = []
segmaps_aug = []
for _ in range(5):
    images_aug_i, segmaps_aug_i = seq(image=image, segmentation_maps=segmap)
    images_aug.append(images_aug_i)
    segmaps_aug.append(segmaps_aug_i)

In [ ]:
# We want to generate an image containing the original input image and
# segmentation maps before/after augmentation. (Both multiple times for
# multiple augmentations.)
#
# The whole image is supposed to have five columns:
# (1) original image,
# (2) original image with segmap,
# (3) augmented image,
# (4) augmented segmap on augmented image,
# (5) augmented segmap on its own in.
#
# We now generate the cells of these columns.
#
# Note that draw_on_image() and draw() both return lists of drawn
# images. Assuming that the segmentation map array has shape (H,W,C),
# the list contains C items.
cells = []
for image_aug, segmap_aug in zip(images_aug, segmaps_aug):
    cells.append(image)                                         # column 1
    cells.append(segmap.draw_on_image(image)[0])                # column 2
    cells.append(image_aug)                                     # column 3
    cells.append(segmap_aug.draw_on_image(image_aug)[0])        # column 4
    cells.append(segmap_aug.draw(size=image_aug.shape[:2])[0])  # column 5

# Convert cells to a grid image and save.
grid_image = ia.draw_grid(cells, cols=5)
imageio.imwrite("example_segmaps.jpg", grid_image)